# Project
This is the main file for project 2 in COMP 472.

**Team:** Deus Ex Machina

**Member(s):** Sobhan Mehrpour Kevishahi - 40122438

**Github Repository :** [https://github.com/Sobhan-M/comp472-project2](https://github.com/Sobhan-M/comp472-project2)

## Implementation Of Algorithms
In this section I will be implementing the different search algorithms as methods. The implemented classes and functions have been included in separate Python files and are used to abstract away tedious aspects of the implementation.

In [1]:
# I realize this way of importing is not good practice, but I know there aren't any conflicts so I'm doing it anyway.

from car import *
from grid import *
from position import *
from node import *
from priorityqueue import *
from reporter import *
from heuristics import *
from iomanager import *

import numpy as np
import time

In [2]:
example1 = "IIB...C.BHHHC.AAD.....D.EEGGGF.....F"
example2 = "C.B...C.BHHHAADD........EEGGGF.....F"
example3 = "...GF...BGF.AABCF....CDD...C....EE.."
example4 = "....F...B.F.AABCF....C.....C....EE.."
example5 = "BBBJCCH..J.KHAAJ.K..IDDLEEI..L....GG H3 K4 J3"

examples = [example1, example2, example3, example4, example5]

### Uniform Cost Search
This is the implementation of uniform cost search. This algorithm only uses the cost (distance from root node in this case) when choosing which node to expand.

In [3]:
def uniformCostSearch(puzzleLine:str, reportNumber=0, outputFileName="ucs", shouldWriteReport=True):
	"""
	Performs uniform cost search to find the solution of the puzzle.
	Creates the solution file and search file, based on the given name.
	Returns the goal node.
	"""

	# Preparing report.
	reporter = Reporter(puzzleLine)
	reporter.startTimer()

	# Setting things up.
	root = generateStartNode(puzzleLine)
	openList = PriorityQueue(lambda n: n.cost()) # The minimizing function is simply the cost.
	openList.insert(root)
	closedList = dict()
	goalNode = None

	# Main loop.
	while not openList.isEmpty():

		visiting = openList.removeMin()
		reporter.countVisit()
		reporter.addToSearchPath(visiting, lambda n: 0, lambda n: n.cost(), lambda n: 0)

		if visiting.isGoal():
			goalNode = visiting
			break
		else:
			closedList[str(visiting)] = visiting

		children = visiting.expandChildren()

		for child in children:
			if closedList.get(str(child)) is not None: # Avoid visiting already visited nodes.
				continue

			if openList.getValue(child) is not None: # Update node if needed.
				if child.cost() < openList.getValue(child).cost():
					openList.updateValue(child)
			else:
				openList.insert(child)

	# Finalizing report.
	reporter.endTimer()
	reporter.setGoalNode(goalNode)
	report = reporter.generateSolutionReport()

	if shouldWriteReport:
		solutionFileName = f"{outputFileName}-sol-{reportNumber}.txt"
		searchFileName = f"{outputFileName}-search-{reportNumber}.txt"

		with open(solutionFileName, "w") as file:
			file.write(report)

		with open(searchFileName, "w") as file:
			file.write("\n".join(reporter.searchPath))

	return goalNode, reporter

In [5]:
uniformCostSearch("IJBBCCIJDDL.IJAAL.EEK.L...KFF..GGHH. F0 G6")

(None, <reporter.Reporter at 0x1fe45abb760>)

In [4]:
# for i in range(5):
# 	uniformCostSearch(examples[i], i+1, "example-reports/ucs-example")

### Greedy Best First Search
In this implementation we use a heuristic to determine which nodes to explore. This does not guarantee an optimal solution.

In [4]:
def greedyBestFirstSearch(puzzleLine:str, heuristic, reportNumber=0, outputFileName="gbfs", shouldWriteReport=True):
	"""
	Performs greedy best first search to find the solution of the puzzle.
	Creates the solution file and search file, based on the given name.
	Returns the goal node.
	"""

	# Preparing report.
	reporter = Reporter(puzzleLine)
	reporter.startTimer()

	# Setting things up.
	root = generateStartNode(puzzleLine)
	openList = PriorityQueue(lambda n: heuristic(n)) # The minimizing function is simply the heuristic.
	openList.insert(root)
	closedList = dict()
	goalNode = None

	# Main loop.
	while not openList.isEmpty():

		visiting = openList.removeMin()
		reporter.countVisit()
		reporter.addToSearchPath(visiting, lambda n: 0, lambda n: 0, lambda n: heuristic(n))

		if visiting.isGoal():
			goalNode = visiting
			break
		else:
			closedList[str(visiting)] = visiting

		children = visiting.expandChildren()

		for child in children:
			if child.isGoal():
				goalNode = child
				break

			if closedList.get(str(child)) is not None: # Avoid visiting already visited nodes.
				continue

			if openList.getValue(child) is not None: # Avoid visiting a child node already in the open list. Heuristics don't change.
				continue

			openList.insert(child)

		if goalNode is not None:
			# Counting the child if it's the goal node.
			reporter.countVisit()
			reporter.addToSearchPath(visiting, lambda n: 0, lambda n: 0, lambda n: heuristic(n))
			break

	# Finalizing report.
	reporter.endTimer()
	reporter.setGoalNode(goalNode)
	report = reporter.generateSolutionReport()

	if shouldWriteReport:
		solutionFileName = f"{outputFileName}-sol-{reportNumber}.txt"
		searchFileName = f"{outputFileName}-search-{reportNumber}.txt"

		with open(solutionFileName, "w") as file:
			file.write(report)

		with open(searchFileName, "w") as file:
			file.write("\n".join(reporter.searchPath))

	return goalNode, reporter

	

In [6]:
# for i in range(5):
# 	greedyBestFirstSearch(examples[i], h1, i+1, "example-reports/gbfs-example")

### Algorithm A Search
This implementation of algorithm A considers both the cost and the heuristic when choosing which node to visit.

In [5]:
def algorithmA(puzzleLine:str, heuristic, reportNumber=0, outputFileName="a", shouldWriteReport=True):
	"""
	Performs greedy best first search to find the solution of the puzzle.
	Creates the solution file and search file, based on the given name.
	Returns the goal node.
	"""

	# Preparing report.
	reporter = Reporter(puzzleLine)
	reporter.startTimer()

	# Setting things up.
	f = lambda n: heuristic(n) + n.cost()
	h = lambda n: heuristic(n)
	g = lambda n: n.cost()

	root = generateStartNode(puzzleLine)
	openList = PriorityQueue(f) # The minimizing function is simply the cost.
	openList.insert(root)
	closedList = dict()
	goalNode = None

	# Main loop.
	while not openList.isEmpty():

		visiting = openList.removeMin()
		reporter.countVisit()
		reporter.addToSearchPath(visiting, f, g, h)

		if visiting.isGoal():
			goalNode = visiting
			break
		else:
			closedList[str(visiting)] = visiting

		children = visiting.expandChildren()

		for child in children:
			
			closedNode = closedList.get(str(child))
			openNode = openList.getValue(child)
			
			if closedNode is not None: # Handling the closed list.
				if f(child) < f(closedNode):
					closedList.pop(str(child))
					openList.insert(child)
				continue
			elif openNode is not None: # Handling open list.
				if f(child) < f(openNode):
					openList.updateValue(child)
				continue
			else:
				openList.insert(child)

	# Finalizing report.
	reporter.endTimer()
	reporter.setGoalNode(goalNode)
	report = reporter.generateSolutionReport()

	if shouldWriteReport:
		solutionFileName = f"{outputFileName}-sol-{reportNumber}.txt"
		searchFileName = f"{outputFileName}-search-{reportNumber}.txt"

		with open(solutionFileName, "w") as file:
			file.write(report)

		with open(searchFileName, "w") as file:
			file.write("\n".join(reporter.searchPath))

	return goalNode, reporter

	

In [8]:
# for i in range(5):
# 	algorithmA(examples[i], h5, i+1, "example-reports/a-example-h5")

## Report Generation
Takes in a file and writes reports for each puzzle. This includes all solution files and search files.

In [6]:
def generateReports(fileName, outputDirectory=""):
	puzzles = extractPuzzleLines(fileName)

	for i in range(len(puzzles)):
		startTime = time.time()
		
		uniformCostSearch(puzzles[i], i, outputDirectory + "ucs")

		greedyBestFirstSearch(puzzles[i], h1, i, outputDirectory + "gbfs-h1")
		greedyBestFirstSearch(puzzles[i], h2, i, outputDirectory + "gbfs-h2")
		greedyBestFirstSearch(puzzles[i], h3, i, outputDirectory + "gbfs-h3")
		greedyBestFirstSearch(puzzles[i], h4, i, outputDirectory + "gbfs-h4")
		greedyBestFirstSearch(puzzles[i], h5, i, outputDirectory + "gbfs-h5")

		algorithmA(puzzles[i], h1, i, outputDirectory + "a-h1")
		algorithmA(puzzles[i], h2, i, outputDirectory + "a-h2")
		algorithmA(puzzles[i], h3, i, outputDirectory + "a-h3")
		algorithmA(puzzles[i], h4, i, outputDirectory + "a-h4")
		algorithmA(puzzles[i], h5, i, outputDirectory + "a-h5")

		endTime = time.time()	
		print(f"Done with item {i} after {endTime - startTime} s.")

In [7]:
generateReports("sample-input.txt", "sample-output/")

Done with item 0 after 0.12494230270385742 s.
Done with item 1 after 19.91180920600891 s.


## Analysis Generation
We can create a function that generates a CSV file that can easily be exported into a spreadsheet for more analysis.

In [11]:
def generateLine(puzzleNum:int, algorithm:str, heuristic:str, reporter:Reporter):
	searchPathLength = reporter.nodesVisited
	executionTime = reporter.timeElapsed()

	if reporter.goalNode is None:
		solutionLength = "N/A"
	else:
		solutionLength = reporter.countTotalMoves()

	return "{},{},{},{},{},{}".format(puzzleNum, algorithm, heuristic, solutionLength, searchPathLength, executionTime)

In [12]:
def generateSpreadsheet(inputFileName, outputFileName):
	puzzles = extractPuzzleLines(inputFileName)
	results = list()
	results.append("Puzzle Number,Algorithm,Heuristic,Length of the Solution,Length of the Search Path,Execution Time (in seconds)")
	
	for i in range(len(puzzles)):
		startTime = time.time()

		reporter1 = uniformCostSearch(puzzles[i], i, shouldWriteReport=False)[1]
		results.append(generateLine(i, "UCS", "N/A", reporter1))

		reporter1 = greedyBestFirstSearch(puzzles[i], h1, i, shouldWriteReport=False)[1]
		reporter2 = greedyBestFirstSearch(puzzles[i], h2, i, shouldWriteReport=False)[1]
		reporter3 = greedyBestFirstSearch(puzzles[i], h3, i, shouldWriteReport=False)[1]
		reporter4 = greedyBestFirstSearch(puzzles[i], h4, i, shouldWriteReport=False)[1]
		reporter5 = greedyBestFirstSearch(puzzles[i], h5, i, shouldWriteReport=False)[1]
		results.append(generateLine(i, "GBFS", "h1", reporter1))
		results.append(generateLine(i, "GBFS", "h2", reporter2))
		results.append(generateLine(i, "GBFS", "h3", reporter3))
		results.append(generateLine(i, "GBFS", "h4", reporter4))
		results.append(generateLine(i, "GBFS", "h5", reporter5))

		reporter1 = algorithmA(puzzles[i], h1, i, shouldWriteReport=False)[1]
		reporter2 = algorithmA(puzzles[i], h2, i, shouldWriteReport=False)[1]
		reporter3 = algorithmA(puzzles[i], h3, i, shouldWriteReport=False)[1]
		reporter4 = algorithmA(puzzles[i], h4, i, shouldWriteReport=False)[1]
		reporter5 = algorithmA(puzzles[i], h5, i, shouldWriteReport=False)[1]
		results.append(generateLine(i, "A/A*", "h1", reporter1))
		results.append(generateLine(i, "A/A*", "h2", reporter2))
		results.append(generateLine(i, "A/A*", "h3", reporter3))
		results.append(generateLine(i, "A/A*", "h4", reporter4))
		results.append(generateLine(i, "A/A*", "h5", reporter5))

		endTime = time.time()	
		print(f"Done with item {i} after {endTime - startTime} s.")

	with open(outputFileName + ".csv", "w") as file:
		file.write("\n".join(results))

In [13]:
generateSpreadsheet("sample-input.txt", "sample-analysis")

Done with item 0 after 0.08552789688110352 s.
